In [ ]:
#Yolov3 Detection Script

import sys
import cv2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from yolo_v3 import Yolo_v3
from utils import load_images, load_class_names, draw_boxes, draw_frame

In [ ]:
_MODEL_SIZE = (416, 416)
_CLASS_NAMES_FILE = r"C:/Users/User/Downloads/coco.names"
_MAX_OUTPUT_SIZE = 20

In [ ]:
def main(input_type, iou_threshold, confidence_threshold, input_names):
    class_names = load_class_names(_CLASS_NAMES_FILE)
    n_classes = len(class_names)

    model = Yolo_v3(n_classes=n_classes, model_size=_MODEL_SIZE,
                    max_output_size=_MAX_OUTPUT_SIZE,
                    iou_threshold=iou_threshold,
                    confidence_threshold=confidence_threshold)
    
    if input_type == 'images':
        batch_size = len(input_names)
        batch = load_images(input_names, model_size=_MODEL_SIZE)
        inputs = tf.placeholder(tf.float32, [batch_size, *_MODEL_SIZE, 3])
        detections = model(inputs, training=False)
        model_vars = tf.global_variables(scope='yolo_v3_model')
        assign_ops = load_weights(model_vars, r"C:/Users/User/Downloads/yolov3.weights/yolov3.weights")

        with tf.Session() as sess:
            sess.run(assign_ops)
            detection_result = sess.run(detections, feed_dict={inputs: batch})

        draw_boxes(input_names, detection_result, class_names, _MODEL_SIZE)

        print('Detections have been saved successfully.')

    elif input_type == 'video':
        inputs = tf.placeholder(tf.float32, [1, *_MODEL_SIZE, 3])
        detections = model(inputs, training=False)
        model_vars = tf.global_variables(scope='yolo_v3_model')
        assign_ops = load_weights(model_vars, r"C:/Users/User/Downloads/yolov3.weights/yolov3.weights")

        with tf.Session() as sess:
            sess.run(assign_ops)

            win_name = 'Video detection'
            cv2.namedWindow(win_name)
            cap = cv2.VideoCapture(input_names[0])
            frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH),
                          cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fourcc = cv2.VideoWriter_fourcc(*'X264')
            fps = cap.get(cv2.CAP_PROP_FPS)
            out = cv2.VideoWriter('./python/Detection.mp4', fourcc, fps,
                                  (int(frame_size[0]), int(frame_size[1])))

            # try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                resized_frame = cv2.resize(frame, dsize=_MODEL_SIZE[::-1],
                                           interpolation=cv2.INTER_NEAREST)
                detection_result = sess.run(detections,
                                            feed_dict={inputs: [resized_frame]})

                draw_frame(frame, frame_size, detection_result,
                           class_names, _MODEL_SIZE)

                cv2.imshow(win_name, frame)
                plt.show()

                key = cv2.waitKey(1) & 0xFF

                if key == ord('q'):
                    break

                out.write(frame)
                    
            print('Detections have been saved successfully.')
            # except:
            #     print('error')
            # finally:
            cv2.destroyAllWindows()
            cap.release()
            out.release()
                

    else:
        raise ValueError("Inappropriate data type. Please choose either 'video' or 'images'.")
        
    tf.reset_default_graph()